In [2]:
from pole_gen.pole_gen import generate_utility_pole_mesh, generate_road_meshes
from utils.plot import plot_open3d
import secrets
import numpy as np

# 2702759322 - Loads of stuff
seed = secrets.randbits(32)
np.random.seed(seed)
print(f"Seed: {seed}")

state = generate_utility_pole_mesh()

# Add some road meshes for debugging
road_meshes = generate_road_meshes(state)
road_meshes.rotate(
    state.geometry.get_rotation_matrix_from_xyz((0, 0, state.z_rotation)),
    center=(0, 0, 0),
)

plot_open3d([state.geometry, road_meshes])

Seed: 3367599841


In [4]:
from pole_gen.raycasting import scan_geometry
from utils.plot import plot_cloud

npoints = 1000

pc = scan_geometry(state, n_points=npoints, sensor_pos=(3, 2, 1), jitter=0.02)
print(f"Number of points: {len(pc.point.positions)}")

plot_cloud(pc)

Number of points: 1000
